In [1]:
import os
import sys
import json
import inspect

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
!export PROJECT_ID="gcp-mira-develop"
project = "gcp-mira-develop"
os.environ["PROJECT_ID"] = project

In [3]:
from common.models import Agent
from common.models.agent import AgentCapability

In [4]:
from services.agents.agent_tools import agent_tool_registry

INFO: [config/config.py:55 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:105 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:106 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:107 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:109 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [langchain_pgvector]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]


In [5]:
agent_tool_registry

{'ruleset_input_tool': StructuredTool(name='ruleset_input_tool', description="ruleset_input_tool(ruleset_name: str) -> dict - Get the list of required inputs to run a set of rules (a 'ruleset').\n  The current available ruleset is a ruleset for medicaid eligibility.\n  The output of this tool is a dict of input keys and corresponding data types.", args_schema=<class 'pydantic.main.ruleset_input_toolSchemaSchema'>, func=<function ruleset_input_tool at 0x12cdae940>),
 'ruleset_execute_tool': StructuredTool(name='ruleset_execute_tool', description='ruleset_execute_tool(ruleset_name: str, rule_inputs: dict) -> dict - Run a business rules engine to make determinations about medicaid\n  eligibility. Takes a dict of constituent attributes as input (such as\n  income level, demographic data etc - the full set of input keys is\n  retrieved using the ruleset_input_tool).  Outputs an eligibility decision.', args_schema=<class 'pydantic.main.ruleset_execute_toolSchemaSchema'>, func=<function rules

In [6]:
from services.agents.agent_service import get_all_agents, get_agent_config

In [7]:
get_agent_config()

{'MediKate': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.agents.RoutingAgent,
  'tools': 'search_tool,query_tool,rules_engine_execute_tool,ruleset_input_tool',
  'capabilities': ['Chat', 'Query', 'Plan', 'Route']},
 'Casey': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.agents.RoutingAgent,
  'tools': 'ALL',
  'datasets': 'fqhc_medical_transactions',
  'capabilities': ['Chat', 'Query', 'Plan', 'Route']},
 'Chat': {'llm_type': 'VertexAI-Chat-Palm2V2-Langchain',
  'agent_type': 'langchain_Conversational',
  'agent_class': services.agents.agents.ChatAgent,
  'capabilities': ['Chat', 'Query']},
 'Task': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_StructuredChatAgent',
  'agent_class': services.agents.agents.TaskAgent,
  'capabilities': ['Chat', 'Query', 'Task']},
 'Plan': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.age

In [9]:
agent_name = "Chat"
agent_params = get_agent_config()[agent_name]
llm_service_agent = agent_params["agent_class"](agent_params["llm_type"], agent_name)

In [14]:
from services.agents.agent_tools import (gmail_tool, docs_tool, database_tool,
                                         google_sheets_tool,
                                         calendar_tool, search_tool,
                                         query_tool)

In [15]:
query_tool

StructuredTool(name='query_tool', description='query_tool(query: str) -> Dict - Perform a query and craft an answer using one of the available query engines.', args_schema=<class 'pydantic.main.query_toolSchemaSchema'>, func=<function query_tool at 0x12cdaed30>)

In [16]:
query_tool.__name__

AttributeError: 'StructuredTool' object has no attribute '__name__'

In [11]:
langchain_agent = llm_service_agent.load_langchain_agent()

TypeError: unhashable type: 'StructuredTool'

In [12]:
llm_service_agent.get_tools()

TypeError: unhashable type: 'StructuredTool'

In [8]:
from services.agents.agents import BaseAgent
from config.utils import get_agent_config

In [9]:
agent_config = get_agent_config()
agent_config

{'MediKate': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.agents.RoutingAgent,
  'tools': 'search_tool,query_tool,rules_engine_execute_tool,ruleset_input_tool',
  'capabilities': ['Chat', 'Query', 'Plan', 'Route']},
 'Casey': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.agents.RoutingAgent,
  'tools': 'ALL',
  'datasets': 'fqhc_medical_transactions',
  'capabilities': ['Chat', 'Query', 'Plan', 'Route']},
 'Chat': {'llm_type': 'VertexAI-Chat-Palm2V2-Langchain',
  'agent_type': 'langchain_Conversational',
  'agent_class': services.agents.agents.ChatAgent,
  'capabilities': ['Chat', 'Query']},
 'Task': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_StructuredChatAgent',
  'agent_class': services.agents.agents.TaskAgent,
  'capabilities': ['Chat', 'Query', 'Task']},
 'Plan': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.age

In [10]:
agent = BaseAgent.load_llm_service_agent("Casey")

In [11]:
agent

In [12]:
agent.get_tools()

[StructuredTool(name='ruleset_input_tool', description="ruleset_input_tool(ruleset_name: str) -> dict - Get the list of required inputs to run a set of rules (a 'ruleset').\n  The current available ruleset is a ruleset for medicaid eligibility.\n  The output of this tool is a dict of input keys and corresponding data types.", args_schema=<class 'pydantic.main.ruleset_input_toolSchemaSchema'>, func=<function ruleset_input_tool at 0x1338ae940>),
 StructuredTool(name='ruleset_execute_tool', description='ruleset_execute_tool(ruleset_name: str, rule_inputs: dict) -> dict - Run a business rules engine to make determinations about medicaid\n  eligibility. Takes a dict of constituent attributes as input (such as\n  income level, demographic data etc - the full set of input keys is\n  retrieved using the ruleset_input_tool).  Outputs an eligibility decision.', args_schema=<class 'pydantic.main.ruleset_execute_toolSchemaSchema'>, func=<function ruleset_execute_tool at 0x1338ae9d0>),
 StructuredT

In [13]:
agent = BaseAgent.load_llm_service_agent("Chat")

In [14]:
agent.get_tools()

[<function services.agents.agent_tools.agent_tool.<locals>.decorator.<locals>.wrapper(*args, **kwargs)>,
 <function services.agents.agent_tools.agent_tool.<locals>.decorator.<locals>.wrapper(*args, **kwargs)>]

In [13]:
from services.agents.agent_service import run_agent, agent_plan

In [ ]:
run_agent("Chat", "how is the weather today?")

In [ ]:
user_id = "5nJrkPWa3D0yCKA853mD"
plan = agent_plan("Plan", "Send an email to my boss asking for a raise", user_id)

In [ ]:
plan[0]

In [ ]:
plan_steps = plan[1].plan_steps

In [ ]:
plan_steps[0].